**Inter-Market Analysis**. In professional trading, this is often called "Lead-Lag" or "Sentiment Correlation."

For example, many traders look at the **RSI of the 10-Year Treasury Yield ($TNX)** or the **US Dollar ($DXY)** to decide whether to buy Tech stocks like Apple. If the Dollar is "Oversold" and bouncing back, Tech usually drops.

To do this in VectorBT, we use **Broadcasting** to pull signals from "Indicator Stocks" and apply them to our "Target Stock."

---

### The Concept: The "Market Guards" Strategy

We will only buy **AAPL** if:

1. **The Tech Sector (QQQ)** is in an uptrend (RSI > 50).
    
2. **The "Fear Gauge" (VIX)** is not overbought (RSI < 60).
    
3. **AAPL itself** is hitting a dip (RSI < 40).
    

---

### The Code: Multi-Stock Logic

The key here is downloading all three symbols and then "mapping" the RSI of the first two onto the price of the third.

In [ ]:
import vectorbt as vbt
import pandas as pd

# 1. Download Target (AAPL) and Guards (QQQ for Sector, ^VIX for Fear)
symbols = ['AAPL', 'QQQ', '^VIX']
data = vbt.YFData.download(symbols, period='2y', missing_columns='drop')
prices = data.get('Close')

# 2. Calculate RSI for all three
# This creates a DataFrame with 3 columns of RSI
all_rsi = vbt.RSI.run(prices, window=14).rsi

# 3. Extract individual RSI series
aapl_rsi = all_rsi['AAPL']
qqq_rsi = all_rsi['QQQ']
vix_rsi = all_rsi['^VIX']

# 4. LOGIC: Using Guards to inform the Target
# We buy AAPL ONLY if QQQ is strong AND the VIX isn't spiking
entry_signal = (qqq_rsi > 50) & (vix_rsi < 60) & (aapl_rsi < 40)
exit_signal = (aapl_rsi > 70)

# 5. PORTFOLIO (Running only on AAPL price)
pf = vbt.Portfolio.from_signals(
    prices['AAPL'], 
    entry_signal, 
    exit_signal, 
    fees=0.001,
    init_cash=10000
)

print("Strategy with Inter-Market Guards:")
print(pf.total_return())

In [ ]:
### Why this is mathematically different:

Instead of looking at a stock in a vacuum, you are creating a **Conditional Filter**.

- **The Sector Filter (`qqq_rsi > 50`):** You are ensuring the "neighborhood" is safe before buying the house.
    
- **The Fear Filter (`vix_rsi < 60`):** You are ensuring there isn't a market-wide panic. When the VIX RSI is very high, correlations go to 1.0, and almost every stock drops regardless of its own RSI.
    

---

### How to Expand This:

VectorBT makes it easy to add more "Guards." You could look at:

- **NVDA RSI:** To see if the AI leader is leading or lagging Apple.
    
- **Bitcoin RSI:** Often used as a proxy for "Risk-On" sentiment.
    
- **Gold RSI:** To see if investors are hiding in "Safe Haven" assets.